In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import sys
sys.path.append('../..')
import utils.tensorflow_preprocess as tp

In [11]:
df = pd.read_csv('../../data/colloquial_correct_train_data.csv')

In [12]:
df

,src,tgt
0,"네, 언제든지 편하실 때 체크아우타시면 도와드릴게요.","네, 언제든지 편하실 때 체크아웃하시면 도와드릴게요."
1,성장하는 재판매 사업짜 그루베 고갱니믈 초대하고 십씀니다.,성장하는 재판매 사업자 그룹에 고객님을 초대하고 싶습니다.
2,"안녕하세요, 예야카려고 전화를 드려써요.","안녕하세요, 예약하려고 전화를 드렸어요."
3,손니미 완는데 방이 업쓰며 너떠캐요?,손님이 왔는데 방이 없으면 어떡해요?
4,"아니요, 시간 낭비예요, 다시느 녀기에 아 놀 꺼예요.","아니요, 시간 낭비예요, 다시는 여기에 안 올 거예요."
...,...,...
49995,어떤 다른 서류가 필요하신지 닶시 말쐼해 주시겠습니까?,어떤 다른 서류가 필요하신지 다시 말씀해 주시겠습니까?
49996,비행기에서 감배를 피울 생각읃 없었어요.,비행기에서 담배를 피울 생각은 없었어요.
49997,전 오래 씂 수 깄는 겍 필요해요.,전 오래 쓸 수 있는 게 필요해요.
49998,털앨 궁화하고 앉았을 땑 반짝이고 푹신한 느낌을 줍니다.,털을 강화하고 앉았을 때 반짝이고 푹신한 느낌을 줍니다.


In [14]:
df['n_src'] = df['src'].apply(lambda x: tp.full_stop_filter(x))
df['n_tgt'] = df['tgt'].apply(lambda x: tp.full_stop_filter(x))

In [15]:
inputs, outputs = tp.tokenize_and_filter(df['n_src'], df['n_tgt'], max_length=24)

In [16]:
print(f'Input shape: {inputs.shape}')
print(f'Output shape: {outputs.shape}')

Input shape: (50000, 24)
Output shape: (50000, 24)


In [17]:
tf_dataset = tp.create_train_dataset(inputs, outputs, batch_size=64, buffer_size=60000)

2022-10-25 14:59:17.648714: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
tf_dataset

<PrefetchDataset element_spec=({'inputs': TensorSpec(shape=(None, 24), dtype=tf.int32, name=None), 'dec_inputs': TensorSpec(shape=(None, 23), dtype=tf.int32, name=None)}, {'outputs': TensorSpec(shape=(None, 23), dtype=tf.int32, name=None)})>

In [19]:
inputs

array([[8226,   23,    2, ...,    0,    0,    0],
       [8226, 5309,    5, ...,    0,    0,    0],
       [8226,   55,    2, ...,    0,    0,    0],
       ...,
       [8226,  303,  919, ...,    0,    0,    0],
       [5591, 8002, 1213, ...,  368,    1, 8227],
       [8226, 3214, 1181, ...,    0,    0,    0]], dtype=int32)

In [2]:
def get_angles(pos, i, d_model):
    '''
    Positional Embedding 의 pos / 10000 ** (2i / d_model) 부분 산출 함수

    PE(pos, 2i) = sin(pos / 10000 ** (2i / d_model))
    PE(pos, 2i + 1) = cos(pos / 10000 ** (2i / d_model))
    '''
    angle_rates = 1 / np.power(10000, (2 * i // 2) / np.float32(d_model))
    # shape = (1, d_model)
    # pos shape :
    # angle_rates shape = (1, d_model)
    # return shape = 행렬곱 (pos, 1), (1, d_model) =  (pos, d_model)
    return np.matmul(pos, angle_rates)

In [3]:
def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)  # shape : (position, d_model)
    # 오른쪽으로 짝수번째 인덱스는 sin 함수를 적용
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    # 오른쪽으로 홀수번째 인덱스는 cos 함수를 적용
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]  # pos_encoding shape : (1, position, d_model)
    # 왜 shape에 1을 추가해주냐면, batch_size 만큼 학습하기 위함임

    return tf.cast(pos_encoding, dtype=tf.float32)


In [4]:
class TransformerEmbedding(tf.keras.layers.Layer):
    def __init__(self, d_model, input_vocab_size, maximum_position_encoding, dropout_rate=0.1):
        super().__init__()

        self.d_model = d_model  # 하나의 단어가 d_model의 차원으로 인코딩 됨
        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        # vocab_size는 tokenizer 내부 vocab.txt의 사이즈
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)  # 포지셔널 인코딩
        self.dropout = tf.keras.layers.Dropout(dropout_rate)  # 드롭아웃 설정

    def __call__(self, x, training):
        # 최초 x의 shape = (batch_size, seq_len)
        seq_len = tf.shape(x)[1]
        out = self.embedding(x)  # shape : (batch_size, input_seq_len, d_model)
        out = out * tf.math.sqrt(
            tf.cast(self.d_model, tf.float32))  # x에 sqrt(d_model) 만큼을 곱해주냐면, 임베딩 벡터보다 포지셔널 인코딩 임베딩 벡터의 영향력을 줄이기 위해서임
        # 포지셔널 인코딩은 순서만을 의미하기 때문에 임베딩 벡터보다 영향력이 적어야 이치에 맞음
        out = out + self.pos_encoding[:, :seq_len, :]
        out = self.dropout(out, training=training)

        return out  # shape : (batch_size, input_seq_len, d_model)

In [5]:
def scaled_dot_product_attention(q, k, v, mask=None):
    # q shape : (batch_size, seq_len, d_model)
    # k shape : (batch_size, seq_len, d_model)
    # v shape : (batch_size, seq_len, d_model)
    matmul_qk = tf.matmul(q, k, transpose_b = True)
    #matmul_qk shape : (batch_size, seq_len, seq_len)

    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # scaled_attetion_logits shape : (batch_size, seq_len, seq_len)

    if mask is not None:
        scaled_attention_logits = scaled_attention_logits + (mask * -1e9)

    softmax = tf.nn.softmax(scaled_attention_logits, axis=-1)

    # softmax shape : (batch_size, seq_len, seq_len)

    output = tf.matmul(softmax, v)

    # output(attention_value) shape : (batch_size, seq_len, d_model)
    # 즉 처음 입력 차원인 (batch_size, seq_len, d_model) 차원을 아웃풋으로 반환
    # 인풋과 아웃풋의 사이즈가 동일하다.

    # scaled_dot_product_attention 의 결과는 단어들 간의 연관성을 학습.
    return output, softmax

In [6]:
class MultiHeadAttention(tf.keras.layers.Layer):
    # 멀티 헤드 어텐션은 전체 어텐션을 분리하여 병렬적으로 어텐션을 수행하는 기법.
    # 이렇게 하는 이유는, 깊은 차원을 한번에 어텐션을 수행하는 것보다, 병렬로 각각 수행하는 것이 더 심도있는 언어들간의 관계를 학습할 수 있기 때문.

    def __init__(self, d_model, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0,2,1,3])

    def __call__(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)


        attention_weights, softmax = scaled_dot_product_attention(q, k, v, mask)

        scaled_attention = tf.transpose(attention_weights, perm=[0,2,1,3])

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

        output = self.dense(concat_attention)

        return output, softmax

In [7]:
class Pointwise_FeedForward_Network(tf.keras.layers.Layer):
    # Pointwise_FeedForward_Network 에서는 인코더의 출력에서 512개의 차원이 2048차원까지 확장되고, 다시 512개의 차원으로 압축된다.
    def __init__(self, d_model, dff):
        super().__init__()
        self.d_model = d_model
        self.dff = dff

        self.middle = tf.keras.layers.Dense(dff, activation='relu')
        self.out = tf.keras.layers.Dense(d_model)

    def __call__(self, x):
        middle = self.middle(x) # middle shape : (batch_size, seq_len, dff)
        out = self.out(middle) # out shape : (batch_size, seq_len, d_model)
        return out
